In [ ]:
!pip install accelerate -U
!pip show accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.8 MB/s eta 0:00:00
Name: accelerate
Version: 0.24.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [ ]:
!pip show transformers
!pip install transformers -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.0
    Uninstalling huggingface-hub-0.19.0:
      Successfully uninstalled huggingface-hub-0.19.0


In [ ]:
!pip install torch transformers


In [ ]:
!pip install transformers accelerate -U


In [ ]:
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

# Function to load data from file
def load_data(file_path):
    texts, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            text, label = line.strip().split(';')
            texts.append(text)
            labels.append(label)
    return texts, labels

# Load data from files
train_texts, train_labels = load_data('/content/train.txt')
test_texts, test_labels = load_data('/content/test.txt')
val_texts, val_labels = load_data('/content/val.txt')

# Initialize tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Prepare datasets
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmotionDataset(train_encodings, train_labels)
val_dataset = EmotionDataset(val_encodings, val_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

# Load pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train and evaluate
trainer.train()

# Evaluate on test data
trainer.evaluate(test_dataset)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.742200
20,1.779500
30,1.757300
40,1.697500
50,1.701800
60,1.746100
70,1.634600
80,1.588700
90,1.537500
100,1.397400


{'eval_loss': 0.16339828073978424,
 'eval_runtime': 5.9001,
 'eval_samples_per_second': 338.979,
 'eval_steps_per_second': 5.424,
 'epoch': 3.0}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # Use 'binary' for binary classification
    return {'accuracy': acc, 'f1': f1}


In [ ]:
# Assuming 'trainer' is your Trainer instance
trainer.compute_metrics = compute_metrics
evaluation_results = trainer.evaluate()


In [ ]:
print("Validation Results:", evaluation_results)


Validation Results: {'eval_loss': 0.1479501575231552, 'eval_accuracy': 0.9375, 'eval_f1': 0.9372006779984146, 'eval_runtime': 7.275, 'eval_samples_per_second': 274.914, 'eval_steps_per_second': 4.399, 'epoch': 3.0}


In [ ]:
model_save_path = '/content/saved_model'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('/content/saved_model/tokenizer_config.json',
 '/content/saved_model/special_tokens_map.json',
 '/content/saved_model/vocab.json',
 '/content/saved_model/merges.txt',
 '/content/saved_model/added_tokens.json',
 '/content/saved_model/tokenizer.json')

In [ ]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/model.safetensors (deflated 12%)
  adding: content/saved_model/config.json (deflated 55%)
  adding: content/saved_model/special_tokens_map.json (deflated 52%)
  adding: content/saved_model/merges.txt (deflated 53%)
  adding: content/saved_model/tokenizer.json (deflated 72%)
  adding: content/saved_model/tokenizer_config.json (deflated 76%)
  adding: content/saved_model/vocab.json (deflated 59%)


In [ ]:
from google.colab import files
files.download('/content/saved_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install flask-ngrok


In [ ]:
!unzip /content/drive/MyDrive/Roberta_Emotion_Analysis_Model.zip

Archive:  /content/drive/MyDrive/Roberta_Emotion_Analysis_Model.zip
   creating: content/saved_model/
  inflating: content/saved_model/model.safetensors  
  inflating: content/saved_model/config.json  
  inflating: content/saved_model/special_tokens_map.json  
  inflating: content/saved_model/merges.txt  
  inflating: content/saved_model/tokenizer.json  
  inflating: content/saved_model/tokenizer_config.json  
  inflating: content/saved_model/vocab.json  


In [ ]:
from flask import Flask, request, jsonify
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
import torch
from flask_ngrok import run_with_ngrok

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# Load model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('/content/content/saved_model')
tokenizer = RobertaTokenizerFast.from_pretrained('/content/content/saved_model')

@app.route('/predict', methods=['POST'])
def predict():
    text = request.json.get("text")
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return jsonify({'class_id': predicted_class_id})

# Run the app
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f8c0-35-238-82-241.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Nov/2023 05:49:06] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2023 05:49:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2023 05:50:11] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2023 05:50:12] "GET /favicon.ico HTTP/1.1" 404 -
